In [2]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import to_categorical

import matplotlib.pyplot as plt
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#constant
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# Now let's do a one-hot encoding and normalize the images:

# convert to categorical
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES)
# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=BATCH_SIZE,
epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT,
verbose=VERBOSE)
score = model.evaluate(X_test, Y_test,
batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
# And the weights learned by our deep network on the training set
model.save_weights('cifar10_weights.h5', overwrite=True)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import numpy as np
NUM_TO_AUGMENT=5
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# augumenting
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i] # (3, 32, 32)
    x = x.reshape((1,) + x.shape) # (1, 3, 32, 32)
    for x_aug in datagen.flow(x, batch_size=1,
        save_to_dir='preview', save_prefix='cifar', save_format='jpeg'):
        if num_aug >= NUM_TO_AUGMENT:
            break
    xtas.append(x_aug[0])
    num_aug += 1
    
#fit the dataget
datagen.fit(X_train)

# train
history = model.fit_generator(datagen.flow(X_train, Y_train,
batch_size=BATCH_SIZE), samples_per_epoch=X_train.shape[0],
epochs=NB_EPOCH, verbose=VERBOSE)
score = model.evaluate(X_test, Y_test,
batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

170498071/170498071 [==============================] - 137s 1us/step
X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                              

This code is an image recognition algorithm from Deep Learning With Keras by Gulli and Pal. The algorithm is trained on the CIFAR-10 dataset that contains 60000 images that span 10 categorical classes (some examples of the categories include: airplane, automobile, bird, cat, deer, etc.) and it aims to train the machine to categorize novel images based on previous learning of attributes shared by other images that category (Gulli & Pal, 2017).

There are ethical and privacy considerations that should be taken into account when working with image recognition AI, especially if the technology is used for more significant uses including facial recognition, for example. Image recognition algorithms can result in privacy concerns when the subject has not explicitly given permission for their image to be used with the technology. If the algorithm was instead trained on faces, permission would be required and the context of the use of the images should be explained to the subject so that they can make an informed decision on whether they wish to participate to comply with GDPR transparency requirements. Specifically, The GDPR requires that software accessed by the public must clearly state the context and intended use of personal information in clear language (Recital 58 - the Principle of Transparency - General Data Protection Regulation (GDPR), 2019).  According to this law, without explicit permission from the subject, private and personally identifying data including photography may breach privacy or copyright laws.

Another ethical issue that arises is the potential for copyright infringement. Images such as photography featuring famous actors or celebrities are often copyrighted. Using copyrighted images requires additional permission and licensing for use by artificial intelligence for these purposes as well. To ensure compliance with copyright laws, open-source images or licensed images should be used in accordance with the license agreement. 

Finally, image recognition algorithms can also pose ethical concerns including discrimination if they are not properly implemented, trained, and maintained (Johnson, 2023). If data is not large and diverse, discriminatory judgments maybe made and inferred by artificial intelligence that categorizes by gender, race, or age, for example (SITNFlash, 2020). AI companies and institutions that use poorly trained algorithms that lead to discriminatory practices may find themselves liable for discriminatory actions that are made on these bases and efforts should always be made to provide holistic and well-rounded data to prevent these inequities in AI. 

**References**


Gulli, A., & Pal, S. (2017). *Deep Learning with Keras.* Packt Publishing Limited. ISBN: 978-1-78712-842-2.

Johnson, A. (2023, May 25). Racism and AI: Here’s how it’s been criticized for amplifying bias. *Forbes.* https://www.forbes.com/sites/ariannajohnson/2023/05/25/racism-and-ai-heres-how-its-been-criticized-for-amplifying-bias/?sh=4b623d1d269d

*Recital 58 - The Principle of Transparency - General Data Protection Regulation (GDPR)*. 
(2019, September 3). General Data Protection Regulation (GDPR). https://gdpr-
info.eu/recitals/no-58/

SITNFlash. (2020, October 26). *Racial discrimination in face recognition technology - Science in the news.* Science in the News. https://sitn.hms.harvard.edu/flash/2020/racial-discrimination-in-face-recognition-technology/#:~:text=Face%20recognition%20algorithms%20boast%20high,and%2018%2D30%20years%20old.